# Setup

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, './venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# Instantiate a brand new model

In [2]:
# tokenizer
from tokenizer import get_tokenizer
tokenizer = get_tokenizer(size = 2048) # size options are 95(character-wise), 128, 256, 512, 1024, 2048 & 4096

# config file
from config import ModelConfig, TrainConfig
cfg = ModelConfig()
cfg.vocab_len = tokenizer.vocab_len
tcfg = TrainConfig()
print(cfg, '\n\n', tcfg)

# model modules
from model import customGPT
model = customGPT(cfg).to(cfg.device)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e3, 'K parameters\n')
print(model)

ModelConfig(dim=64, vocab_len=2051, device='cpu', num_layers=10, second_resid_norm=False, mlp_hidden_mult=1, mlp_bias=False, mlp_nonlinearity='SiLU', mlp_gated=True, num_q_heads=10, num_kv_heads=2, head_dim=16, theta=10000, max_seq_len=512, scale_first_resid=True, norm_type='RMSNorm', norm_affine=True, norm_bias=True, eps=1e-06, max_batch_size=1) 

 TrainConfig(weight_decay=0.02, batch_size=32, max_iters=1000, eval_interval=10, eval_samples=1, checkpoint_interval=None, lr_max=0.1, lr_min=1e-06, warmup_iters=100, final_flat_iters=100, anneal_type='cos', num_restarts=3, T_mult=2)
502.592 K parameters

customGPT(
  (token_embedder): Embedding(2051, 64)
  (layers): ModuleList(
    (0-9): 10 x ResidualLayer(
      (pre_attn_norm): Norm()
      (attn): MQSA(
        (Wq): Linear(in_features=64, out_features=160, bias=False)
        (Wk): Linear(in_features=64, out_features=32, bias=False)
        (Wv): Linear(in_features=64, out_features=32, bias=False)
        (Wo): Linear(in_features=160, 

# Training

In [3]:
import torch
from tools import get_data_loader
from train import scheduler_lambda, train

optimizer = torch.optim.AdamW(model.parameters(), lr = tcfg.lr_max, weight_decay = tcfg.weight_decay)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=scheduler_lambda)

train_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='train')
test_data_loader = get_data_loader(batch_size=tcfg.batch_size, split='validation')

In [4]:
if False: # set to true if you'd like to see a graph of the learning rate schedule
    import matplotlib.pyplot as plt
    
    # Generate learning rate values
    lrs = [scheduler_lambda(i) for i in range(tcfg.max_iters)]
    
    # Plot the learning rates
    plt.figure(figsize=(10, 5))
    plt.plot(lrs, label='Learning Rate')
    plt.title('Learning Rate Schedule')
    plt.xlabel('Iteration')
    plt.ylabel('Learning Rate')
    plt.grid(True)
    plt.legend()
    plt.show()

In [4]:
model, optimizer, log_data = train(
    model, 
    tokenizer, 
    cfg, 
    optimizer,
    scheduler,
    tcfg, 
    train_data_loader,
    test_data_loader,
    #log_data: list = None, 
    #detect_anomoly = False # use if you're getting crazy errors about a the gradient being broken
)

  0%|                                                                          | 1/1000 [00:20<5:42:40, 20.58s/it]

step 0000: lr 0.000000, train loss 63.9337, val loss 64.2233, ppl 7794836105729155934864801792, time elapsed: 9.98 seconds


  1%|▊                                                                        | 11/1000 [02:10<3:05:26, 11.25s/it]

step 0010: lr 0.001000, train loss 61.9759, val loss 61.7167, ppl 635625584866842491603648512, time elapsed: 121.95 seconds


  2%|█▌                                                                       | 21/1000 [03:36<2:44:37, 10.09s/it]

...

step 0980: lr 0.000000, train loss 2.8420, val loss 2.7914, ppl 16, time elapsed: 7817.49 seconds


 99%|███████████████████████████████████████████████████████████████████████▎| 991/1000 [2:11:44<01:23,  9.29s/it]

step 0990: lr 0.000000, train loss 2.7034, val loss 2.7596, ppl 16, time elapsed: 7897.30 seconds


100%|███████████████████████████████████████████████████████████████████████| 1000/1000 [2:12:55<00:00,  7.98s/it]

step 0999: lr 0.000000, train loss 2.8573, val loss 2.6890, ppl 15, time elapsed: 7967.83 seconds


# inference test before you decide to save it
if `tcfg.checkpoint_interval != None` then checkpoints have already been saved

In [5]:
from inference import generate
prompt = "Once upon a time"
model.eval()
output = generate(
    prompt, 
    model, 
    tokenizer,
    #max_gen_len = 512,
    temperature = 0.7,
    #memory_saver_div = 8,
    #top_p = 0.9,
    #top_k = 32,
)
model.train()
print(output)

Once upon a time, there was a little boy named Tim. Tim was a loved in a big buse. One day, the dog very Lily in the park. The bird saw a big ball of the were very out not dog. They looked to fious with the balk. The dog was so happy. It was started to play with a like to the caf. The cat was a slide would not toy lose is sk. The dog was not pick of a lose of the made all the care. They were happy for the The bird went for a moused to the clostil. The little water and happy and the bird. The little girl and said, "I have went of the was not man her friends, and we collower ain the good to ted and in the dirl for the rung and Tim was something the ran and in the win, and made the fly.
The wind a fun, and were happy and they more a on her.


# Saving your final model
you DO still need to do this even if you had been saving checkpoints; the final state has not yet been saved

In [6]:
from tools import save_model
save_model(model, cfg, tcfg, log_data)